In [1]:
import os, sys
target = "AMP-Algo-Trading"
while os.path.basename(os.getcwd()) != target:
    os.chdir("..")
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())

In [2]:
from SQLite_Manager.sqlManager import SqlAlchemyDataBaseManager
import sqlite3
import pandas as pd
import os

In [3]:
directory = r"data/stock_market_data/sp500/csv"
path = r"backtesterClass"
dbName = "stockDB"

db = SqlAlchemyDataBaseManager(f"{path}/{dbName}.db")
# Walk through the directory
for root, dirs, files in os.walk(directory):
    for i, file in enumerate(files):  # Use enumerate to track iteration count
        try :
            df = pd.read_csv(directory+"/"+file)
            db.update(f"{file[:-4]}",df)
        except:
            print(file)


/Users/elyazidbenkhadra/Desktop/LEO/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql(table_name, con=self.conn.connection, if_exists="replace")
/Users/elyazidbenkhadra/Desktop/LEO/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql(table_name, con=self.conn.connection, if_exists="replace")
/Users/elyazidbenkhadra/Desktop/LEO/AMP-Algo-Trading/SQLite_Manager/sqlManager.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchem

In [14]:
db_path = r"backtesterClass/stockDB.db"  
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
# start_date, end_date = "01-01-2022", "12-12-2022"
# Get all table names (tickers)
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
tickers = [t[0] for t in tables]  


df_ini = pd.read_sql(f"SELECT Date, Close FROM {tickers[0]}", conn)
df_ini['Date'] = pd.to_datetime(df_ini['Date'], format="%d-%m-%Y")

for i,ticker in enumerate(tickers[1:]):
    try:
        df = pd.read_sql(f"SELECT Date, Close FROM {ticker}", conn)
        df['Date'] = pd.to_datetime(df['Date'], format="%d-%m-%Y")

        if i == 0:
            merged_df = pd.merge_asof(df_ini.sort_values('Date'), 
                                df.sort_values('Date'), 
                                on='Date', 
                                direction='backward',
                                suffixes=(f'_{tickers[0]}', f'_{ticker}')) 
        else:
            merged_df = pd.merge_asof(merged_df.sort_values('Date'), 
                                df.sort_values('Date'), 
                                on='Date', 
                                direction='backward',
                                suffixes=(f'_{tickers[i]}', f'_{ticker}'))
            
    except:
        print(ticker)
        pass

GS-PJ
BF-A
MS-PF
STZ-B


In [15]:
merged_df.dropna(axis=1)

,Date,Close_CSCO,Close_TROW,Close_NVR,Close_MRO,Close_BA,Close_MDT,Close_MO,Close_SWKS,Close_CDNS,...,Close_SWK,Close_MU,Close_TRV,Close_AEP,Close_JNJ,Close_WM,Close_DOV,Close_IEX,Close_LRCX,Close_NWL
0,1990-02-16,0.077257,1.750000,4.500000,8.698958,20.958332,1.898438,12.750000,1.000000,5.055556,...,17.625000,1.025000,14.625000,30.250000,6.937500,1.375000,6.137761,2.222222,1.416667,12.3750
1,1990-02-20,0.079861,1.734375,4.500000,8.636823,20.500000,1.906250,12.583333,1.000000,4.916667,...,17.375000,0.987500,14.625000,29.750000,6.812500,1.375000,6.116813,2.246914,1.361111,11.8125
2,1990-02-21,0.078125,1.718750,4.750000,8.543620,20.166668,1.929688,12.541667,1.000000,4.833333,...,17.687500,1.025000,14.625000,29.750000,6.718750,1.375000,6.116813,2.197531,1.416667,11.7500
3,1990-02-22,0.078993,1.734375,4.750000,8.574688,20.208332,1.953125,12.708333,0.916667,4.972222,...,17.500000,1.112500,14.750000,29.875000,6.640625,1.375000,6.116813,2.296296,1.444444,11.5000
4,1990-02-23,0.078559,1.703125,4.500000,8.792161,20.000000,1.921875,12.500000,0.916667,4.916667,...,17.625000,1.137500,14.687500,29.875000,6.515625,1.375000,6.074917,2.296296,1.500000,11.8125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8264,2022-12-06,48.590000,121.330002,4600.000000,27.450001,178.429993,77.080002,46.209999,92.239998,163.660004,...,77.730003,53.680000,188.500000,96.370003,176.100006,165.559998,139.770004,233.779999,443.089996,12.7500
8265,2022-12-07,48.180000,121.250000,4725.000000,27.410000,176.500000,76.910004,46.369999,91.959999,160.589996,...,77.250000,53.820000,188.050003,96.669998,177.169998,165.210007,138.350006,235.050003,444.279999,12.8100
8266,2022-12-08,48.990002,122.260002,4694.279785,26.840000,179.080002,77.790001,46.860001,95.169998,164.059998,...,78.269997,55.200001,187.720001,97.720001,177.199997,166.009995,137.259995,234.139999,455.000000,13.3000
8267,2022-12-09,48.459999,119.830002,4648.529785,26.170000,179.539993,78.589996,46.709999,94.519997,163.669998,...,79.230003,54.869999,185.649994,96.580002,175.740005,166.830002,137.289993,235.210007,450.200012,13.4400
